# BEST AVAILABLE PIXEL COMPOSITE (BAP) *in Google Earth Engine Python API*
based on *Pixel-Based Image Compositing for Large-Area Dense
Time Series Applications and Science (White, 2014)*
https://goo.gl/Fi8fCY

## The process consist in 2 steps:
1. Compute all scores
2. Make composite

## 1. Compute all scores
The first step is to build a `Bap` object using this package:

### Make imports

In [5]:
# Import Earth Engine and initialize
import ee
ee.Initialize()

# Import packages
import geebap
from geetools import tools,collection, __version__
import ipygee as ui
import pprint

print('geebap version', geebap.__version__)
print('geetools version', __version__)

geebap version 0.2.3
geetools version 0.4.0


### The `Bap` object will be created using some parameters:

``` python
bap = geebap.Bap(season, range, colgroup, scores, masks, filters, target_collection, brdf, harmonize)
```
As follows:

## Season
This object holds information of the growing season (start and end). This object does not hold any year, just day and month. For example, start on November 15 (11-15) and ends on March 15 (03-15)

In [6]:
a_season = geebap.Season('11-15', '03-15')

You can add the year to it

In [7]:
daterange = a_season.add_year(2019)

In [8]:
ui.eprint(daterange)

Note that when the season covers two years, start date is in the previous year.

## Range

This is a 2 items `tuple` indicating the amount of years that will be used. The first item indicates how many years to go "backwards" and the second "forward"

In [32]:
a_range = (0, 0) # One year backwards

## Colgroup
This comes from `geetools.collection.CollectioGroup` and basically aggregates the collections available in that module.

You could also make your own `colgroup`, but you have to keep in mind that it is composed by `geetools.Collection` objects.

If this parameter is `None` the process will use the `priority` module, that holds information in which Landsat satellite was available according to the year parsed.

In [10]:
# My own colgroup
l8toa = collection.Landsat8TOA()
l7toa = collection.Landsat7TOA()
mycolgroup = collection.CollectionGroup(l8toa, l7toa)

## Masks
The mask that will be used to get rid of `clouds`, `shadow`, etc

In [11]:
cld_mask = geebap.masks.Mask()

## Filters
There are two filters you can use in the process:

**cloud percentage**: `filters.CloudsCover`

**masked pixel percentage**: `filters.MaskCover`. This filter can be used **only** if maskpercent score is included in the process.

In [12]:
filt_cld = geebap.filters.CloudCover() # defaults on 70 %
filt_mask = geebap.filters.MaskCover() # defaults on 70 %

## Scores
This is what makes the difference. Every score has its own parameters, and share the following:

- **range_out**: the range of values the score will be, by default it is (0, 1)
- **name**: the name of the resulting band

Also, all scores have a static method that can be used without the need of a `Bap` object. I'd be the core method of every score. In the scores that need a whole `ee.ImageCollection` to be computed, this method is called `apply` and the ones that can be computed using a single `ee.Image` the method is called `compute`

## White's scores

### DOY (best day of the year)
Basically, pixels from images closer to that date will have higher score
It takes 2 params:

- **best day of the year**: You have to specify which day you are goint to prioritize
- **season**: You can use the same as the one for the process, or not

Optional:

- **function**: There are two options: `linear` or `gauss`. Defaults to `linear`

In [13]:
doy = geebap.scores.Doy('01-15', a_season)

### Satellite
It uses a list of available satellite for each year that you can check:

In [14]:
# List of satellites for 2000
geebap.priority.SeasonPriority.relation[2000]

['LANDSAT/LE07/C01/T1_SR',
 'LANDSAT/LE07/C01/T1_TOA',
 'LANDSAT/LT05/C01/T1_SR',
 'LANDSAT/LT05/C01/T1_TOA']

the score has one main param:

**ratio**: how much score will decrease for each satellite. For example, for 2000, if rate is 0.05 (default value):

* 'LANDSAT/LE07/C01/T1_SR' --> 1
* 'LANDSAT/LE07/C01/T1_TOA' --> 0.95
* 'LANDSAT/LT05/C01/T1_SR' --> 0.90
* 'LANDSAT/LT05/C01/T1_TOA' --> 0.85

In [15]:
sat = geebap.scores.Satellite()

### Atmospheric Opacity
It uses the atmospheric opacity band computed by Surface Reflectance, so only SR collections will have this score. If the process uses a non-SR collection, like TOA, the this score will be zero.

In [16]:
atm_op = geebap.scores.AtmosOpacity()

### Distance to mask
This assigns a score regarding the distance of the pixel to the closest masked pixel. As the only mask is for clouds, it could be considered 'distance to cloud'. It has 3 main params:

**unit**: unit to measure distance. Defaults to 'meters'

**dmax**: max distance. Pixel further than this distance will have score 1. Defaults to 600

**dmin**: min distance. Defaults to 0 (next pixel from the maks will have score 0).

In [17]:
dist = geebap.scores.CloudDist()

## Custom Scores *(not White's)*

### Outliers
It computes an statistics over the whole collection (in the season) and assigns score regarding the *distance* of each pixel value to that statistic. It has 3 main parameters:

**bands**: a list of bands to include in the process. The process splits the score in the number of given bands. For example, if 4 bands are given, the max score for each band will be 0.25

**process**: one of 'mean' or 'median'. Defaults to 'median'

**dist**: distance from 'mean' or 'median'. Defaults to 0.7

*NOTE: bands must be in the image, so if you use a vegetation index, be sure to include it in the process*

In [18]:
out = geebap.scores.Outliers(("ndvi",))

### Mask percentage
It computes the precentage of masked pixels in the image (not the whole scene). It has one main parameter:

**band**: the band that holds the mask.

In [19]:
maskper = geebap.scores.MaskPercent()

### Mask percentage (Kernel)
Similar to `MaskPercent` but uses a kernel for computation. It may help to avoid exceed memory capacity and also may help to build bigger composites

In [20]:
maskper_kernel = geebap.scores.MaskPercentKernel()

### Vegetation Index
This scores is based on the absolute value of the given index, parametrized to `range_out`. The only parameter is **index**: the name of it (*ndvi*, *evi* or *nbr*). Defaults to *ndvi*.

In [21]:
ind = geebap.scores.Index("ndvi")

### Multiple years (seasons)
If you want to use images from a range of seasons, and not just only one, this scores prioritizes the main season. Take in count that a season may hold 2 years, but the main is the least (see `Season`). It has 3 main params:

**main_year**: this is the central year. Images from this year (season) will have score 1

**season**: a `Season` object.

**ratio**: amount of score that will decrease as it goes further to the main year. Defaults to 0.05. It is similar to *rate* parameter in `Satellite`.



In [22]:
# Will not use it in the test
multi = geebap.scores.MultiYear(2019, a_season, range_out=(0.8, 1))

## Target collection
The `target_collection` parameter of the `Bap` object defines the output's band type. It must be an instance of `geetools.Collection` and defaults to `collection.Landsat8SR()`. So, by default, the values for the bands of the output will be parametrized between 0 and 10000, except for the new bands

## Making the composite (BAP)
Next step is to create a `Bap` object. It has the following parameters:

In [33]:
bap_obj = geebap.Bap(
    range=a_range,
    season=a_season,
    masks=(cld_mask,),
    scores=(
        doy, 
        sat, 
        # atm_op, 
        dist,
        out, 
        # maskper,
        maskper_kernel, # use one at a time
        ind,
        multi,
    ),
    filters=(
        filt_cld, 
        # filt_mask
    ),
    brdf=True,
    harmonize=True
)

## Define a site

In [34]:
site = ee.Geometry.Polygon([[-71.5,-42.5],
                            [-71.5,-43],
                            [-72,-43],
                            [-72,-42.5]])

## Finally, compute the composite
`Bap` object has a method named `build_composite_best` that creates one image out of the pixels with higher score in the collection (which includes all collections given). It has the following parameters:

- **year**:  The year to process
- **site**: The site where the composite will be computed
- **indices**: a list of vegetation indices. Defaults to `None`
- **name**: name for the band that holds the final score. Defaults to *score*
- **buffer**: distance of the buffer around the site. Defaults to 0.
- **add_individual_scores**: if this param is `True` the resulting composite will have all individual scores

In [35]:
composite = bap_obj.build_composite_best(2019, site, indices=['ndvi'], add_individual_scores=True)

## You can also get the complete collection in which all images will have the computed scores

In [26]:
bapcol = bap_obj.compute_scores(2019, site, indices=['ndvi'], add_individual_scores=True)

## Inspect the results in a Map

In [27]:
Map = ui.Map()

In [28]:
Map.show()

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …

In [29]:
vis = bap_obj.target_collection.visualization('NSR', True)

In [31]:
Map.addLayer(site)

In [36]:
Map.addLayer(composite, vis, 'BAP composite (0,0)')

In [23]:
Map.addLayer(bapcol.select(['nir', 'swir', 'red', '^score.+', 'date', 'col_id']), vis, 'BAP collection')

In [24]:
Map.centerObject(composite)

In [25]:
Map.addLayer(composite.select('date').randomVisualizer(), dict(min=0, max=255), 'Composite dates')